# Análise dos dados de COVID-19 no Brasil

## Importando bibliotecas

In [ ]:
#!pip install plotly==4.1.0
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Criando funções úteis para o projeto

In [ ]:
def formata_dataframe(dados: pd.DataFrame) -> pd.DataFrame:
    tradutor: dict = {
        'Province/State' : 'Provincia/Estado',
        'Country/Region' : 'Pais/Regiao'
    }
    
    dados.rename(columns = tradutor, inplace = True)
    dados.drop(columns = ['Lat', 'Long'], inplace = True)
    
def dados_recentes_por_pais(dados: pd.DataFrame) -> pd.DataFrame:
    ultima_data = dados.columns[-1]
    return dados.groupby('Pais/Regiao')[ultima_data].sum()


## Abrindo as bases de dados dos casos confirmados, óbitos e casos recuperados

In [ ]:
casos_confirmados = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
casos_obitos = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
casos_recuperados = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")


In [ ]:
formata_dataframe(casos_confirmados)
formata_dataframe(casos_obitos)
formata_dataframe(casos_recuperados)

In [ ]:
display(casos_confirmados.head())
display(casos_obitos.head())
display(casos_recuperados.head())

## Casos no Brasil na última data registrada

In [ ]:
ultima_data = casos_confirmados.columns[-1]
print(f"O número de casos confirmados no Brasil em {ultima_data} foi de: {dados_recentes_por_pais(casos_confirmados)['Brazil']}")
print(f"O número de recuperados no Brasil em {ultima_data} foi de: {dados_recentes_por_pais(casos_recuperados)['Brazil']}")
print(f"O número de óbitos no Brasil em {ultima_data} foi de: {dados_recentes_por_pais(casos_obitos)['Brazil']}")

## Criando o novo DataFrame unificado para os dados do Brasil 

In [ ]:
casos_confirmados_Brasil = casos_confirmados.groupby('Pais/Regiao').sum().loc[['Brazil']]
casos_obitos_Brasil = casos_obitos.groupby('Pais/Regiao').sum().loc[['Brazil']]
casos_recuperados_Brasil = casos_recuperados.groupby('Pais/Regiao').sum().loc[['Brazil']]

In [ ]:
print(f'O número de dados nulos na base de casos confirmados é: {casos_confirmados_Brasil.isna().sum().sum()}')
print(f'O número de dados nulos na base de óbitos: {casos_obitos_Brasil.isna().sum().sum()}')
print(f'O número de dados nulos na base de recuperados: {casos_recuperados_Brasil.isna().sum().sum()}')

In [ ]:
display(casos_confirmados_Brasil.head())
display(casos_obitos_Brasil.head())
display(casos_recuperados_Brasil.head())

In [ ]:
total_Brasil = pd.concat([casos_confirmados_Brasil, casos_obitos_Brasil, casos_recuperados_Brasil])
total_Brasil.index =  ['Confirmados', 'Obitos', 'Recuperados']
total_Brasil.rename_axis('Brasil', inplace = True)
total_Brasil.head()

## Preparando os dados para plotagem

A primeira data em que houve registro de casos do novo coronavírus foi: 26/02/2020

In [ ]:
casos_Brasil_plot = total_Brasil.T.loc['2/26/20':]
casos_Brasil_plot

### Tipo de dado no índice

In [ ]:
casos_Brasil_plot.index = pd.to_datetime(casos_Brasil_plot.index)
print(f'Tipo de dado no índice: {casos_Brasil_plot.index.dtype}')

## Plotagem dos casos confirmados, óbitos e casos recuperados com Plotly

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = casos_Brasil_plot.index, 
                         y = casos_Brasil_plot['Confirmados'], 
                         name = 'Casos Confirmados'))

fig.add_trace(go.Scatter(x = casos_Brasil_plot.index, 
                         y = casos_Brasil_plot['Obitos'], 
                         name = 'Óbitos'))

fig.add_trace(go.Scatter(x = casos_Brasil_plot.index, 
                         y = casos_Brasil_plot['Recuperados'], 
                         name = 'Recuperados'))
fig.update_layout(title = 'Avanço do COVID-19 no Brasil')

## Crescimento dos casos confirmados, óbitos e casos recuperados no Brasil em 2021

In [ ]:
total_Brasil = total_Brasil.T
total_Brasil.head()

In [ ]:
total_Brasil['Diferenca Confirmados'] = total_Brasil['Confirmados'].diff()
total_Brasil['Diferenca Obitos'] = total_Brasil['Obitos'].diff()
total_Brasil['Diferenca Recuperados'] = total_Brasil['Recuperados'].diff()
total_Brasil.tail()

In [ ]:
brasil_2021 = total_Brasil.loc['1/1/21':]

### Função para plotagem dos crescimentos

In [ ]:
def plota_crescimento_media_movel(df: pd.DataFrame, coluna: str, titulo: str, yaxis_nome: str):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = df.index, 
                             y = df[coluna], 
                             name = 'Variação'))
    fig.add_trace(go.Scatter(x = df.index, 
                             y = df[coluna].rolling(7, center = True).mean(), 
                             name = 'Média Móvel Semanal'))
    fig.update_layout(title = titulo,
                      yaxis_title = yaxis_nome)
    
    return fig.show()

###  Variação dos casos confirmados no Brasil

In [ ]:
plota_crescimento_media_movel(brasil_2021, 'Diferenca Confirmados', 
                              'Crescimento dos casos confirmados de COVID-19 no Brasil', 
                              'Variação dos Casos Confirmados')

###  Variação dos óbitos no Brasil

In [ ]:
plota_crescimento_media_movel(brasil_2021, 'Diferenca Obitos', 
                              'Crescimento dos óbitos por COVID-19 no Brasil', 
                              'Variação dos Óbitos')

###  Variação dos casos recuperados no Brasil

In [ ]:
plota_crescimento_media_movel(brasil_2021, 'Diferenca Recuperados', 
                              'Crescimento dos casos recuperados de COVID-19 no Brasil', 
                              'Variação dos Casos Recuperados')

## Taxa de letalidade no Brasil

Calculo da taxa de letalidade 1: Óbitos/Confirmados.

Calculo da taxa de letalidade 2: Óbitos/(Recuperados + Óbitos).

In [ ]:
taxa_de_letalidade_1 = pd.eval('brasil_2021.Obitos / brasil_2021.Confirmados  * 100')
taxa_de_letalidade_2 = pd.eval('brasil_2021.Obitos / (brasil_2021.Obitos + brasil_2021.Recuperados) * 100')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = brasil_2021.index, 
                         y = taxa_de_letalidade_1, 
                         name = 'Taxa de Letalidade 1'))
fig.add_trace(go.Scatter(x = brasil_2021.index, 
                         y = taxa_de_letalidade_2, 
                         name = 'Taxa de Letalidade 2'))
fig.update_layout(title = 'Variação das Taxas de Letalidade ao longo de 2021')
fig.show()

## Avanço da vacinação no Brasil

### Abrindo a base de dados e realizando um tratamento prévio

In [ ]:
vacinacao = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/country_data/Brazil.csv')
vacinacao.drop(columns = ['location', 'source_url'], inplace = True)
tradutor = {
    'date' : 'Data',
    'vaccine' : 'Vacina',
    'total_vaccinations' : 'Vacinação Total',
    'people_vaccinated' : 'Pessoas Vacinadas Primeira Dose',
    'people_fully_vaccinated' : 'Pessoas Vacinadas Todas as Doses'
}
vacinacao.rename(columns = tradutor, inplace = True)
vacinacao['Data'] = pd.DatetimeIndex(vacinacao['Data'])
vacinacao = vacinacao.set_index('Data')
vacinacao.head()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = vacinacao['Vacinação Total'], 
                         name = 'Vacinação Total'))

fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = vacinacao['Pessoas Vacinadas Primeira Dose'], 
                         name = 'Pessoas Vacinadas Primeira Dose'))

fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = vacinacao['Pessoas Vacinadas Todas as Doses'], 
                         name = 'Pessoas Vacinadas Todas as Doses'))
fig.update_layout(title = 'Avanço da Vacinação Contra COVID-19 no Brasil')
fig.show()

### Plotagem de todos os dados sobre o novo coronavírus

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = vacinacao['Vacinação Total'], 
                         name = 'Vacinação Total'))

fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = vacinacao['Pessoas Vacinadas Primeira Dose'], 
                         name = 'Pessoas Vacinadas Primeira Dose'))

fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = vacinacao['Pessoas Vacinadas Todas as Doses'], 
                         name = 'Pessoas Vacinadas Todas as Doses'))
fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = casos_Brasil_plot['Confirmados'][vacinacao.index[0]:], 
                         name = 'Casos Confirmados'))

fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = casos_Brasil_plot['Obitos'][vacinacao.index[0]:], 
                         name = 'Óbitos'))

fig.add_trace(go.Scatter(x = vacinacao.index, 
                         y = casos_Brasil_plot['Recuperados'][vacinacao.index[0]:], 
                         name = 'Recuperados'))
fig.update_layout(title = 'Avanço do COVID-19 e da imunização no Brasil')

###  Variação da Vacinação Total

In [ ]:
vacinacao['Variação da Vacinação Total'] = vacinacao['Vacinação Total'].diff()
vacinacao.head()

In [ ]:
plota_crescimento_media_movel(vacinacao, 'Variação da Vacinação Total', 
                              'Crescimento da Vacinação Contra COVID-19 no Brasil', 
                              'Variação da Vacinação Total')

# Previsões de dados

## Importando bibliotecas

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from datetime import datetime

## Tratamento prévio dos dados

Primeiro caso de COVID-19 foi registrado no Brasil no dia 26 de fevereiro de 2020  

In [ ]:
dataset = total_Brasil['2/26/20':]

In [ ]:
dataset.head()

#### Alterando o tipo de dado do índice

In [ ]:
print(f'O tipo de dado do índice do dataset é: {dataset.index.dtype}')

In [ ]:
dataset.index = pd.to_datetime(dataset.index)

In [ ]:
print(f'O tipo de dado do índice do dataset é: {dataset.index.dtype}')

### Função para análise dos erros dos modelos

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def verifica_erro(coluna_original: str, coluna_previsto: str, data: pd.DataFrame,
                  nome_coluna: str = '', nome_indice: str = '') -> pd.DataFrame:
    
    vies = np.mean(data[coluna_original] - data[coluna_previsto])
    mse = mean_squared_error(data[coluna_original], data[coluna_previsto])
    rmse = np.sqrt(mean_squared_error(data[coluna_original], data[coluna_previsto]))
    mae = mean_absolute_error(data[coluna_original], data[coluna_previsto])
    mape = np.mean(np.abs((data[coluna_original] - data[coluna_previsto]) / data[coluna_original])) * 100
    
    lista_dos_erros = [vies, mse, rmse, mae, mape]
    erros = pd.DataFrame(lista_dos_erros, index = ['VIÉS','MSE','RMSE','MAE', 'MAPE'], 
                         columns = [nome_coluna])
    erros.rename_axis(nome_indice, inplace = True)
    
    return erros

## Modelo de Média Móvel Exponencial para os dados de casos confirmados

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
dataset['Diferenca Confirmados'].head()

### Plotando os dados 

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = dataset.index, 
                         y = dataset['Diferenca Confirmados'].values, 
                         name = 'Variação dos Casos Confirmados'))
fig.update_layout(title = 'Crescimento dos casos confirmados de COVID-19 no Brasil')

### Estatísticas de estacionariedade dos dados

In [ ]:
resultado_confirmados = adfuller(dataset['Diferenca Confirmados'].values)
print(f'Estatística ADF: {resultado_confirmados[0]}')
print(f'p-valor: {resultado_confirmados[1]}')

In [ ]:
sns.set_style('darkgrid')
decomposicao_confirmados = seasonal_decompose(dataset['Diferenca Confirmados'], period = 35)
plt.rcParams['figure.figsize'] = (12, 6)
decomposicao_confirmados.plot();

### Dividindo o dataset em treino e teste

In [ ]:
divisor = int(len(dataset['Diferenca Confirmados']) * 0.9)

x = dataset[['Diferenca Confirmados']]

x_treino = x[: divisor].copy()
x_treino.index =  pd.DatetimeIndex(x_treino.index).to_period('D')

x_teste = x[divisor :].copy()
x_teste.index =  pd.DatetimeIndex(x_teste.index).to_period('D')

In [ ]:
print(f'A quantidade de dados nulos na base de treino: {x_treino.isna().sum().sum()}')
print(f'A quantidade de dados nulos na base de teste: {x_teste.isna().sum().sum()}')
print(f'Shape da base de treino: {x_treino.shape}')
print(f'Shape da base de teste: {x_teste.shape}')

### Treinando o modelo definido

In [ ]:
alpha = 0.8

modelo = ExponentialSmoothing(x_treino['Diferenca Confirmados']).fit(smoothing_level = alpha)
x_treino['Previsto'] = modelo.predict(start = 0, end = len(x_treino) - 1)

### Realizando a previsão para a base de teste

In [ ]:
historico = [x_treino.iloc[i, 0] for i in range(len(x_treino))]
historico_previsao = [x_treino.iloc[i, 1] for i in range(len(x_treino))]
previsao = []
for t in range(len(x_teste)):
    y_pred = historico_previsao[-1] + alpha * (historico[-1] - historico_previsao[-1])
    y_real = x_teste.iloc[t, 0]
    previsao.append(y_pred)
    historico.append(y_real)
    historico_previsao.append(y_pred)

x_teste['Previsão'] = previsao

In [ ]:
x_teste.head()

### Verificação dos erros do modelo exposto

In [ ]:
verifica_erro('Diferenca Confirmados', 'Previsão', x_teste, 'Exponential Smoothing', 'Base Teste')

In [ ]:
x_treino.index = x_treino.index.to_timestamp()
x_teste.index = x_teste.index.to_timestamp()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = x_treino.index, 
                         y = x_treino['Diferenca Confirmados'].values, 
                         name = 'X real'))
fig.add_trace(go.Scatter(x = x_teste.index, 
                         y = x_teste['Diferenca Confirmados'].values, 
                         name = 'Y real'))
fig.add_trace(go.Scatter(x = x_teste.index, 
                         y = x_teste['Previsão'].values, 
                         name = 'Y previsto'))

### Finalizando o Modelo

In [ ]:
x.head()

In [ ]:
alpha = 0.8
confirmados_reais = x.copy()
confirmados_reais.index = pd.DatetimeIndex(confirmados_reais.index).to_period('D')

modelo = ExponentialSmoothing(confirmados_reais['Diferenca Confirmados']).fit(smoothing_level = alpha)
confirmados_reais['Previsto'] = modelo.predict(start = 0, end = len(confirmados_reais) - 1)
confirmados_reais.head()

In [ ]:
dia_seguinte = confirmados_reais.iloc[-1, 1] + alpha * (confirmados_reais.iloc[-1, 0] - confirmados_reais.iloc[-1, 1])

In [ ]:
print(f'Confirmados em {confirmados_reais.index[-1]}: {round(confirmados_reais.iloc[-1, 0])}')
print(f'Previsão de confirmados em {confirmados_reais.index[-1]}: {round(confirmados_reais.iloc[-1, 1])}')
print(f'Previsão de confirmados em amanhã ({confirmados_reais.index[-1] + 1}): {round(dia_seguinte)}')